## Récupération des prix CAL

In [1]:
# Packages nécessaires
import pandas as pd
import os
import glob

In [2]:
# Chemin d'accès
data = r"C:\Users\Meryam GHASSIR\UPLE Dropbox\Achat & Appro\SAUVEGARDE OUTILS\Calcul-Portefeuille\P50\Injection\test-models\Consommation\data\EEX"
output = r"C:\Users\Meryam GHASSIR\UPLE Dropbox\Achat & Appro\SAUVEGARDE OUTILS\Calcul-Portefeuille\P50\Injection\test-models\Consommation\Output"

In [ ]:
# Vérification de la structure des dossiers (test sur dossier 2022)
year_2022_path = os.path.join(data, "2022")
if os.path.exists(year_2022_path):
    print(f"Dossier 2022 existe: {year_2022_path}")
    
    # Lister les sous-dossiers
    subdirs = [d for d in os.listdir(year_2022_path) if os.path.isdir(os.path.join(year_2022_path, d))]
    print(f"Sous-dossiers dans 2022: {len(subdirs)}")
    print(f"Premiers sous-dossiers: {sorted(subdirs)[:5]}")
    
    # Vérifier le contenu d'un sous-dossier
    if subdirs:
        first_subdir = subdirs[0]
        subdir_path = os.path.join(year_2022_path, first_subdir)
        files_in_subdir = os.listdir(subdir_path)
        print(f"\nContenu du dossier {first_subdir}:")
        print(files_in_subdir)
        
        # Vérifier s'il y a des fichiers PowerFutureResults
        power_files = [f for f in files_in_subdir if f.startswith('PowerFutureResults')]
        print(f"Fichiers PowerFutureResults trouvés: {power_files}")
else:
    print(f"❌ Dossier 2022 non trouvé: {year_2022_path}")

# Essayer avec un pattern plus spécifique
print("\n=== RECHERCHE AVEC PATTERN CORRIGÉ ===")
pattern_test = os.path.join(data, "2022", "*", "PowerFutureResults_FR_*.csv")
files_found = glob.glob(pattern_test)
print(f"Pattern utilisé: {pattern_test}")
print(f"Fichiers trouvés: {len(files_found)}")

if files_found:
    print(f"Premier fichier: {files_found[0]}")
    
    # Essayer de lire le premier fichier pour diagnostiquer
    try:
        with open(files_found[0], 'r', encoding='utf-8') as f:
            first_lines = f.readlines()[:10]
        print("\nPremières lignes du fichier:")
        for i, line in enumerate(first_lines):
            print(f"Ligne {i+1}: {repr(line.strip())}")
    except Exception as e:
        print(f"Erreur lors de la lecture: {e}")

Dossier 2022 existe: C:\Users\Meryam GHASSIR\UPLE Dropbox\Achat & Appro\SAUVEGARDE OUTILS\Calcul-Portefeuille\P50\Injection\test-models\Consommation\data\EEX\2022
Sous-dossiers dans 2022: 257
Premiers sous-dossiers: ['20220103', '20220104', '20220105', '20220106', '20220107']

Contenu du dossier 20220103:
['FutureStylePowerOptionResults_FR_20220103.csv', 'PowerFutureResults_FR_20220103.csv', 'PowerOptionResults_FR_20220103.csv']
Fichiers PowerFutureResults trouvés: ['PowerFutureResults_FR_20220103.csv']

=== RECHERCHE AVEC PATTERN CORRIGÉ ===
Pattern utilisé: C:\Users\Meryam GHASSIR\UPLE Dropbox\Achat & Appro\SAUVEGARDE OUTILS\Calcul-Portefeuille\P50\Injection\test-models\Consommation\data\EEX\2022\*\PowerFutureResults_FR_*.csv
Fichiers trouvés: 257
Premier fichier: C:\Users\Meryam GHASSIR\UPLE Dropbox\Achat & Appro\SAUVEGARDE OUTILS\Calcul-Portefeuille\P50\Injection\test-models\Consommation\data\EEX\2022\20220103\PowerFutureResults_FR_20220103.csv

Premières lignes du fichier:
Ligne 1

In [3]:
# Lecture de tous les fichiers par année et par dossier
cal_data_all = {}

for year in ['2022', '2023', '2024']:
    pattern = os.path.join(data, year, "*", "PowerFutureResults_FR_*.csv")
    csv_files = glob.glob(pattern)
    
    print(f"\n=== ANNÉE {year} ===")
    print(f"Pattern: {pattern}")
    print(f"Fichiers trouvés: {len(csv_files)}")
    
    if len(csv_files) == 0:
        # Essayer de comprendre pourquoi aucun fichier n'est trouvé
        year_path = os.path.join(data, year)
        if os.path.exists(year_path):
            subdirs = [d for d in os.listdir(year_path) if os.path.isdir(os.path.join(year_path, d))]
            print(f"  Sous-dossiers trouvés: {len(subdirs)}")
            if subdirs:
                # Vérifier le premier sous-dossier
                first_subdir_path = os.path.join(year_path, subdirs[0])
                files_in_first = os.listdir(first_subdir_path)
                print(f"  Fichiers dans {subdirs[0]}: {files_in_first}")
        continue
    
    for file_path in csv_files:
        try:
            folder_name = os.path.basename(os.path.dirname(file_path))
            
            # Diagnostic du fichier avant lecture
            print(f"Tentative de lecture: {folder_name}")
            
            # Essayer plusieurs méthodes de lecture
            df = None
            methods = [
                lambda: pd.read_csv(file_path),
                lambda: pd.read_csv(file_path, sep=';'),
                lambda: pd.read_csv(file_path, encoding='utf-8-sig'),
                lambda: pd.read_csv(file_path, encoding='latin1'),
                lambda: pd.read_csv(file_path, skiprows=1),
                lambda: pd.read_csv(file_path, on_bad_lines='skip') # Cette méthode qui a fonctionné (méthode 6)
            ]
            
            for i, method in enumerate(methods):
                try:
                    df = method()
                    if not df.empty:
                        print(f"OK {folder_name} (méthode {i+1})")
                        break
                except Exception as method_error:
                    if i == 0:  # Afficher l'erreur de la première méthode seulement
                        print(f"  Erreur méthode 1: {str(method_error)[:50]}...")
                    continue
            
            if df is not None and not df.empty:
                cal_data_all[f"{year}_{folder_name}"] = df
            else:
                print(f"KO {folder_name}: Toutes les méthodes ont échoué")
            
        except Exception as e:
            print(f"KO {folder_name}: {e}")

print(f"\nTotal général: {len(cal_data_all)} fichiers")

# Il doit y avoir 768 fichiers au total


=== ANNÉE 2022 ===
Pattern: C:\Users\Meryam GHASSIR\UPLE Dropbox\Achat & Appro\SAUVEGARDE OUTILS\Calcul-Portefeuille\P50\Injection\test-models\Consommation\data\EEX\2022\*\PowerFutureResults_FR_*.csv
Fichiers trouvés: 257
Tentative de lecture: 20220103
  Erreur méthode 1: Error tokenizing data. C error: Expected 1 fields ...
OK 20220103 (méthode 6)
Tentative de lecture: 20220104
  Erreur méthode 1: Error tokenizing data. C error: Expected 1 fields ...
OK 20220104 (méthode 6)
Tentative de lecture: 20220105
  Erreur méthode 1: Error tokenizing data. C error: Expected 1 fields ...
OK 20220105 (méthode 6)
Tentative de lecture: 20220106
  Erreur méthode 1: Error tokenizing data. C error: Expected 1 fields ...
OK 20220106 (méthode 6)
Tentative de lecture: 20220107
  Erreur méthode 1: Error tokenizing data. C error: Expected 1 fields ...
OK 20220107 (méthode 6)
Tentative de lecture: 20220110
  Erreur méthode 1: Error tokenizing data. C error: Expected 1 fields ...
OK 20220110 (méthode 6)
Ten

In [4]:
# Analyser la structure des données CAL

# Vérifier la répartition par année
years_count = {}
for key in cal_data_all.keys():
    year = key.split('_')[0]
    years_count[year] = years_count.get(year, 0) + 1

print("Répartition par année :")
for year, count in sorted(years_count.items()):
    print(f"  {year}: {count} fichiers CSV")

# Analyser un fichier type
first_key = list(cal_data_all.keys())[0]
sample_df = cal_data_all[first_key]

print(f"\nExemple d'un fichier CSV ({first_key}):")
print(f"Shape: {sample_df.shape}")
print(f"Colonnes: {list(sample_df.columns)}")
print("\nContenu:")
print(sample_df.head())

# Vérifier si tous les fichiers ont la même structure
print(f"\nVérification de la cohérence des {len(cal_data_all)} fichiers:")
all_shapes = [df.shape for df in cal_data_all.values()]
all_columns = [list(df.columns) for df in cal_data_all.values()]

print(f"Nombre de shapes différents: {len(set(all_shapes))}")
print(f"Exemple de shapes: {list(set(all_shapes))[:5]}")

Répartition par année :
  2022: 257 fichiers CSV
  2023: 255 fichiers CSV
  2024: 256 fichiers CSV

Exemple d'un fichier CSV (2022_20220103):
Shape: (199, 1)
Colonnes: ['# Prices/Volumes of EEX Power Future Market France']

Contenu:
  # Prices/Volumes of EEX Power Future Market France
0                                                  #
1         # Data type(ST);Trading Date;Creation Time
2  # Data type(PR);Product;Long Name;Maturity;Del...
3  # Data type(OT);Product;Long Name;Maturity;Del...
4                    # Data type(AL);Number of Lines

Vérification de la cohérence des 768 fichiers:
Nombre de shapes différents: 113
Exemple de shapes: [(135, 1), (124, 1), (190, 1), (137, 1), (192, 1)]


In [5]:
# On retraite une deuxième fois afin que le séparateur soit pris en compte dans la lecture de fichier
cal_data_fixed = {}

def process_eex_file(file_path):
    """Traite un fichier EEX PowerFutureResults avec la bonne structure"""
    try:
        # Lire toutes les lignes
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        # Trouver la ligne d'en-têtes PR (ligne 4)
        pr_header_line = None
        for i, line in enumerate(lines):
            if line.strip().startswith('# Data type(PR);'):
                pr_header_line = line.strip()[2:]  # Enlever '# '
                break
        
        if pr_header_line:
            # Extraire les noms de colonnes
            column_names = pr_header_line.split(';')
            
            # Trouver où commencent les vraies données (première ligne avec PR)
            data_start_line = None
            for i, line in enumerate(lines):
                if line.strip().startswith('PR;'):
                    data_start_line = i
                    break
            
            if data_start_line is not None:
                # Lire uniquement les données à partir de cette ligne
                df = pd.read_csv(file_path, 
                                sep=';', 
                                skiprows=data_start_line,  # Ignorer jusqu'aux données PR
                                header=None)  # Pas d'en-tête dans les données
                
                # Appliquer les noms de colonnes de la ligne 4
                if df.shape[1] <= len(column_names):
                    df.columns = column_names[:df.shape[1]]
                else:
                    # Si plus de colonnes que prévu, ajouter des noms génériques
                    extra_cols = df.shape[1] - len(column_names)
                    df.columns = column_names + [f'extra_col_{i}' for i in range(extra_cols)]
                
                # Filtrer uniquement les lignes PR
                df_pr = df[df['Data type(PR)'] == 'PR'].copy()
                
                return df_pr, column_names
            
    except Exception as e:
        print(f"Erreur: {e}")
        return None, None
    
    return None, None

In [6]:
# Test sur quelques fichiers d'abord (test ok - cellule du dessous pour traitement de l'ensemble des fichiers)
for key in list(cal_data_all.keys())[:3]:
    year, date_folder = key.split('_', 1)
    
    # Chemin du fichier
    pattern = os.path.join(data, year, date_folder, "PowerFutureResults_FR_*.csv")
    files = glob.glob(pattern)
    
    if files:
        file_path = files[0]
        df_processed, columns = process_eex_file(file_path)
        
        if df_processed is not None and not df_processed.empty:
            print(f"{key}: {df_processed.shape}")
            print(f"  Colonnes: {list(df_processed.columns)[:5]}...")
            
            # Conversion des colonnes de prix
            price_columns = ['Open Price', 'High Price', 'Low Price', 'Last Price', 'Settlement Price']
            for col in price_columns:
                if col in df_processed.columns:
                    # Remplacer virgule par point et convertir en numérique
                    df_processed[col] = df_processed[col].astype(str).str.replace(',', '.')
                    df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
            
            # Ajouter métadonnées
            df_processed['year'] = year
            df_processed['date_folder'] = date_folder
            df_processed['trading_date'] = f"{year}-{date_folder[4:6]}-{date_folder[6:8]}"
            
            cal_data_fixed[key] = df_processed
            
            # Afficher un aperçu
            print(f"  Aperçu:")
            print(df_processed.head(2))
        else:
            print(f"{key}: Échec du traitement")

print(f"\nFichiers traités: {len(cal_data_fixed)}")

2022_20220103: (138, 23)
  Colonnes: ['Data type(PR)', 'Product', 'Long Name', 'Maturity', 'Delivery Start']...
  Aperçu:
  Data type(PR) Product                           Long Name Maturity  \
0            PR    F701  EEX French Power Base Day 1 Future  2022-01   
1            PR    F702  EEX French Power Base Day 2 Future  2022-01   

  Delivery Start Delivery End  Open Price Timestamp Open Price  High Price  \
0     2022-01-01   2022-01-01         NaN                  NaN         NaN   
1     2022-01-02   2022-01-02         NaN                  NaN         NaN   

  Timestamp High Price  ...  Lot Size Traded Lots  Number of Trades  \
0                  NaN  ...      24.0         NaN               NaN   
1                  NaN  ...      24.0         NaN               NaN   

  Traded Volume  Open Interest Lots Open Interest Volume  Unit of Volumes  \
0           NaN               200.0               4800.0              MWh   
1           NaN               200.0               4800.0  

In [28]:
# Traitement de tous les fichiers
cal_data_final = {}

for key in cal_data_all.keys():
    year, date_folder = key.split('_', 1)
    
    # Chemin du fichier
    pattern = os.path.join(data, year, date_folder, "PowerFutureResults_FR_*.csv")
    files = glob.glob(pattern)
    
    if files:
        file_path = files[0]
        
        try:
            # Lire le fichier ligne par ligne
            with open(file_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()
            
            # Trouver la ligne d'en-têtes (ligne 4)
            header_line = None
            data_start = None
            
            for i, line in enumerate(lines):
                if line.strip().startswith('# Data type(PR);'):
                    header_line = line.strip()[2:]  # Enlever '# '
                elif line.strip().startswith('PR;'):
                    data_start = i
                    break
            
            if header_line and data_start is not None:
                # Extraire les noms de colonnes
                column_names = header_line.split(';')
                
                # Lire les données à partir de la ligne PR
                df = pd.read_csv(file_path, 
                                sep=';', 
                                skiprows=data_start, 
                                header=None)
                
                # Appliquer les noms de colonnes
                df.columns = column_names[:df.shape[1]]
                
                # Garder uniquement les lignes PR
                df_pr = df[df.iloc[:, 0] == 'PR'].copy()  # Première colonne = type
  
                # Ajouter métadonnées (pour repérer de quel fichier provient les données)
                df_pr['year'] = year
                df_pr['date_folder'] = date_folder
                    
                cal_data_final[key] = df_pr
            
        except Exception as e:
            print(f"{key}: {str(e)[:50]}...")

print(f"\nTotal traité: {len(cal_data_final)} fichiers")

# Vérifier un échantillon
if cal_data_final:
    sample_key = list(cal_data_final.keys())[0]
    sample_df = cal_data_final[sample_key]
    
    print(f"\n=== ÉCHANTILLON ({sample_key}) ===")
    print(f"Shape: {sample_df.shape}")
    print(f"Colonnes: {list(sample_df.columns)}")
    print("\nAperçu des données:")
    print(sample_df.head())
    
    # Vérifier les colonnes de prix
    price_cols = [col for col in sample_df.columns if 'Price' in col]
    if price_cols:
        print(f"\nColonnes de prix: {price_cols}")
        print(sample_df[price_cols].head())



Total traité: 768 fichiers

=== ÉCHANTILLON (2022_20220103) ===
Shape: (138, 25)
Colonnes: ['Data type(PR)', 'Product', 'Long Name', 'Maturity', 'Delivery Start', 'Delivery End', 'Open Price', 'Timestamp Open Price', 'High Price', 'Timestamp High Price', 'Low Price', 'Timestamp Low Price', 'Last Price', 'Timestamp Last Price', 'Settlement Price', 'Unit of Prices', 'Lot Size', 'Traded Lots', 'Number of Trades', 'Traded Volume', 'Open Interest Lots', 'Open Interest Volume', 'Unit of Volumes', 'year', 'date_folder']

Aperçu des données:
  Data type(PR) Product                           Long Name Maturity  \
0            PR    F701  EEX French Power Base Day 1 Future  2022-01   
1            PR    F702  EEX French Power Base Day 2 Future  2022-01   
2            PR    F703  EEX French Power Base Day 3 Future  2022-01   
3            PR    F704  EEX French Power Base Day 4 Future  2022-01   
4            PR    F705  EEX French Power Base Day 5 Future  2022-01   

  Delivery Start Delivery 

In [29]:
# Vérification des données présentes dans la colonne 'Settlement Price'
for key, df in cal_data_final.items():
    if 'Settlement Price' in df.columns:
        print(f"{key}: Settlement Price - {df['Settlement Price'].isna().sum()} NaN, {df['Settlement Price'].notna().sum()} valides")
    else:
        print(f"{key}: Settlement Price non trouvée")

2022_20220103: Settlement Price - 45 NaN, 93 valides
2022_20220104: Settlement Price - 43 NaN, 85 valides
2022_20220105: Settlement Price - 41 NaN, 85 valides
2022_20220106: Settlement Price - 38 NaN, 86 valides
2022_20220107: Settlement Price - 36 NaN, 86 valides
2022_20220110: Settlement Price - 45 NaN, 93 valides
2022_20220111: Settlement Price - 43 NaN, 85 valides
2022_20220112: Settlement Price - 41 NaN, 85 valides
2022_20220113: Settlement Price - 39 NaN, 85 valides
2022_20220114: Settlement Price - 36 NaN, 86 valides
2022_20220117: Settlement Price - 45 NaN, 93 valides
2022_20220118: Settlement Price - 43 NaN, 85 valides
2022_20220119: Settlement Price - 41 NaN, 85 valides
2022_20220120: Settlement Price - 37 NaN, 87 valides
2022_20220121: Settlement Price - 35 NaN, 87 valides
2022_20220124: Settlement Price - 42 NaN, 96 valides
2022_20220125: Settlement Price - 40 NaN, 88 valides
2022_20220126: Settlement Price - 38 NaN, 88 valides
2022_20220127: Settlement Price - 36 NaN, 88 v

In [32]:
# Filtrage et nettoyage - on garde que les colonnes nécessaires + suppression des lignes avec Settlement Price NaN

# Colonnes à conserver
colonnes_a_garder = ['Long Name', 'Maturity', 'Delivery Start', 'Settlement Price']

cal_data_clean = {}

for key, df in cal_data_final.items():
    print(f"\nTraitement de {key}...")
    print(f"  Colonnes disponibles: {list(df.columns)}")
    
    # Vérifier que toutes les colonnes existent
    colonnes_disponibles = [col for col in colonnes_a_garder if col in df.columns]
    colonnes_manquantes = [col for col in colonnes_a_garder if col not in df.columns]
    
    if colonnes_manquantes:
        print(f"{key}: Colonnes manquantes - {colonnes_manquantes}")
    
    if colonnes_disponibles:
        print(f" Colonnes trouvées: {colonnes_disponibles}")

        # Créer un nouveau DataFrame avec seulement les colonnes nécessaires
        df_filtered = df[colonnes_disponibles + ['year', 'date_folder']].copy()

        # Conversion de date_folder en datetime
        df_filtered['date_folder'] = pd.to_datetime(df_filtered['date_folder'], errors='coerce')
        
        # Nettoyer les données
        # Supprimer les lignes où Settlement Price est NaN ou vide
        # if 'Settlement Price' in df_filtered.columns:
        # df_filtered['Settlement Price'] = pd.to_numeric(df_filtered['Settlement Price'], errors='coerce')
        df_filtered = df_filtered.dropna(subset=['Settlement Price'])
            # df_filtered = df_filtered[df_filtered['Settlement Price'] != 'NaN']
        
        # 3. Nettoyer Maturity
        if 'Maturity' in df_filtered.columns:
            df_filtered['Maturity'] = pd.to_datetime(df_filtered['Maturity'])
        
        # 4. Convertir Delivery Start en date si possible
        if 'Delivery Start' in df_filtered.columns:
                df_filtered['Delivery Start'] = pd.to_datetime(df_filtered['Delivery Start'])

        cal_data_clean[key] = df_filtered
        
        print(f"{key}: {df_filtered.shape} - Colonnes: {list(df_filtered.columns)}")
    else:
        print(f"{key}: Aucune colonne utilisable trouvée")

print(f"\nFichiers nettoyés: {len(cal_data_clean)}")

# Vérifier un échantillon nettoyé
if cal_data_clean:
    sample_key = list(cal_data_clean.keys())[0]
    sample_df = cal_data_clean[sample_key]
    
    print(f"\n=== ÉCHANTILLON NETTOYÉ ({sample_key}) ===")
    print(f"Shape: {sample_df.shape}")
    print(f"Colonnes: {list(sample_df.columns)}")
    print("\nAperçu:")
    print(sample_df)
    
    print(f"\nTypes de données:")
    print(sample_df.dtypes)


Traitement de 2022_20220103...
  Colonnes disponibles: ['Data type(PR)', 'Product', 'Long Name', 'Maturity', 'Delivery Start', 'Delivery End', 'Open Price', 'Timestamp Open Price', 'High Price', 'Timestamp High Price', 'Low Price', 'Timestamp Low Price', 'Last Price', 'Timestamp Last Price', 'Settlement Price', 'Unit of Prices', 'Lot Size', 'Traded Lots', 'Number of Trades', 'Traded Volume', 'Open Interest Lots', 'Open Interest Volume', 'Unit of Volumes', 'year', 'date_folder']
 Colonnes trouvées: ['Long Name', 'Maturity', 'Delivery Start', 'Settlement Price']
2022_20220103: (93, 6) - Colonnes: ['Long Name', 'Maturity', 'Delivery Start', 'Settlement Price', 'year', 'date_folder']

Traitement de 2022_20220104...
  Colonnes disponibles: ['Data type(PR)', 'Product', 'Long Name', 'Maturity', 'Delivery Start', 'Delivery End', 'Open Price', 'Timestamp Open Price', 'High Price', 'Timestamp High Price', 'Low Price', 'Timestamp Low Price', 'Last Price', 'Timestamp Last Price', 'Settlement Pric

In [38]:
# Nettoyage de Settlement Price (présence de données non-numériques à convertir)
for key, df in cal_data_clean.items():
    if 'Settlement Price' in df.columns:
        # Remplacer virgule par point et convertir en numérique
        df['Settlement Price'] = df['Settlement Price'].astype(str).str.replace(',', '.')
        df['Settlement Price'] = pd.to_numeric(df['Settlement Price'], errors='coerce')
    else:
        print(f"{key}: Settlement Price non trouvée")


In [40]:
# Vérification des données présentes dans la colonne 'Settlement Price'
for key, df in cal_data_clean.items():
    if 'Settlement Price' in df.columns:
        print(f"{key}: Settlement Price - {df['Settlement Price'].isna().sum()} NaN, {df['Settlement Price'].notna().sum()} valides")
    else:
        print(f"{key}: Settlement Price non trouvée")

2022_20220103: Settlement Price - 0 NaN, 93 valides
2022_20220104: Settlement Price - 0 NaN, 85 valides
2022_20220105: Settlement Price - 0 NaN, 85 valides
2022_20220106: Settlement Price - 0 NaN, 86 valides
2022_20220107: Settlement Price - 0 NaN, 86 valides
2022_20220110: Settlement Price - 0 NaN, 93 valides
2022_20220111: Settlement Price - 0 NaN, 85 valides
2022_20220112: Settlement Price - 0 NaN, 85 valides
2022_20220113: Settlement Price - 0 NaN, 85 valides
2022_20220114: Settlement Price - 0 NaN, 86 valides
2022_20220117: Settlement Price - 0 NaN, 93 valides
2022_20220118: Settlement Price - 0 NaN, 85 valides
2022_20220119: Settlement Price - 0 NaN, 85 valides
2022_20220120: Settlement Price - 0 NaN, 87 valides
2022_20220121: Settlement Price - 0 NaN, 87 valides
2022_20220124: Settlement Price - 0 NaN, 96 valides
2022_20220125: Settlement Price - 0 NaN, 88 valides
2022_20220126: Settlement Price - 0 NaN, 88 valides
2022_20220127: Settlement Price - 0 NaN, 88 valides
2022_2022012

In [41]:
# Concaténer tous les DataFrames nettoyés
df_concat = pd.concat(cal_data_clean.values(), ignore_index=True)

# Vérifier le résultat du regroupement
print(df_concat)

# Sauvegarder le DataFrame final
df_concat.to_csv(os.path.join(output, "prix_cal_22_24.csv"), index=False) # fichier ok

                                  Long Name   Maturity Delivery Start  \
0        EEX French Power Base Day 1 Future 2022-01-01     2022-01-01   
1        EEX French Power Base Day 2 Future 2022-01-01     2022-01-02   
2        EEX French Power Base Day 3 Future 2022-01-01     2022-01-03   
3        EEX French Power Base Day 4 Future 2022-01-01     2022-01-04   
4        EEX French Power Base Day 5 Future 2022-01-01     2022-01-05   
...                                     ...        ...            ...   
53785  EEX French Power Peak Quarter Future 2026-10-01     2026-10-01   
53786  EEX French Power Peak Quarter Future 2027-01-01     2027-01-01   
53787     EEX French Power Peak Year Future 2026-01-01     2026-01-01   
53788     EEX French Power Peak Year Future 2027-01-01     2027-01-01   
53789     EEX French Power Peak Year Future 2028-01-01     2028-01-03   

       Settlement Price  year date_folder  
0                 87.08  2022  2022-01-03  
1                 49.12  2022  2022

In [42]:
# Vérification du nombre de lignes par year et Long Name
verif = df_concat.groupby(['year', 'Long Name']).size().reset_index(name='count')
print(verif)


     year                               Long Name  count
0    2022      EEX French Power Base Day 1 Future     62
1    2022     EEX French Power Base Day 10 Future     64
2    2022     EEX French Power Base Day 11 Future     63
3    2022     EEX French Power Base Day 12 Future     64
4    2022     EEX French Power Base Day 13 Future     64
..    ...                                     ...    ...
279  2024  EEX French Power Peak Weekend 2 Future     17
280  2024  EEX French Power Peak Weekend 3 Future     21
281  2024  EEX French Power Peak Weekend 4 Future     22
282  2024  EEX French Power Peak Weekend 5 Future      5
283  2024       EEX French Power Peak Year Future   1022

[284 rows x 3 columns]


In [43]:
# Regrouper par Long Name, année et maturité pour avoir une ligne unique par combinaison (long name et maturité)
df_grouped = df_concat.groupby(['Long Name', 'year', 'Maturity']).agg({
    'Settlement Price': ['mean', 'median','count', 'std', 'min', 'max'],
    'date_folder': ['min', 'max']
}).round(4)

# Aplatir les colonnes multi-index
df_grouped.columns = ['prix_moyen', 'prix_median', 'nb_observations', 'ecart_type', 'prix_min', 'prix_max', 'date_debut', 'date_fin']
df_grouped = df_grouped.reset_index()

print(df_grouped.head())
print(f"Nombre de lignes: {len(df_grouped)}")

df_grouped.to_csv(os.path.join(output, "prix_cal_moyen.csv"), index=False) # fichier ok

                            Long Name  year   Maturity  prix_moyen  \
0  EEX French Power Base Day 1 Future  2022 2022-01-01      87.080   
1  EEX French Power Base Day 1 Future  2022 2022-02-01     221.986   
2  EEX French Power Base Day 1 Future  2022 2022-03-01     282.804   
3  EEX French Power Base Day 1 Future  2022 2022-04-01     320.998   
4  EEX French Power Base Day 1 Future  2022 2022-05-01     183.170   

   prix_median  nb_observations  ecart_type  prix_min  prix_max date_debut  \
0       87.080                1         NaN     87.08     87.08 2022-01-03   
1      223.720                5      7.8563    213.76    229.69 2022-01-26   
2      270.020                5     38.9266    250.95    350.70 2022-02-23   
3      317.500                5     14.2935    307.82    345.50 2022-03-28   
4      185.805                6     11.7742    168.67    194.74 2022-04-25   

    date_fin  
0 2022-01-03  
1 2022-02-01  
2 2022-03-01  
3 2022-04-01  
4 2022-05-02  
Nombre de lignes: 31